# JuMP でたくさん user-defined functions を使う時のはなし

JuliaのJuMPで `register` をつかって自分の定義した関数をモデルの中で使い時、関数がたくさんあったり、N個の関数を使いたかったりしたらどうするかっていうのをまとめました。
ここで考える最小化問題はこれ↓

\begin{align*}
    \min &\quad x_1^2 + x_2^2 \\
    \text{subject to} 
    &\quad x_1 + x_2 \le 2 \\
    &\quad - x_1 + x_2 \le 1 \\
    &\quad - x_1 - x_2 \le -1 \\
    &\quad x_1 - x_2 \le 1
\end{align*}

答えは $(x_1, x_2) = (0.5, 0.5)$。
（これくらいなら `register` 使わないでもできるんだけど、demonstrationということで。）

In [22]:
using JuMP
using Ipopt
using Printf

## Pattern 1

一番単純（？）な方法。
たくさん関数があると大変だし、コードを書く時点で関数の数がわからなかったりするときに使えない。

In [56]:
model = Model(Ipopt.Optimizer);
set_silent(model)

@variable(model, x[i = 1:2]);

register(model, :fn1, 2, (x...) -> x[1] + x[2] - 2.0; autodiff = true);
register(model, :fn2, 2, (x...) -> - x[1] + x[2] - 1.0; autodiff = true);
register(model, :fn3, 2, (x...) -> - x[1] - x[2] + 1.0 ; autodiff = true);
register(model, :fn4, 2, (x...) -> x[1] - x[2] - 1.0; autodiff = true);

@NLconstraint(model, fn1(x...) <= 0.0);
@NLconstraint(model, fn2(x...) <= 0.0);
@NLconstraint(model, fn3(x...) <= 0.0);
@NLconstraint(model, fn4(x...) <= 0.0);

@NLobjective(model, Min, x[1]^2 + x[2]^2);

print(model)
optimize!(model)

objective_min = objective_value(model);
minx = value.(x)

@printf("Objective function value: %.7f \n", objective_min)
@printf("Minimizing values: %.7f, %.7f", minx[1], minx[2])

if (minx[1] ≈ 0.5) & (minx[2] ≈ 0.5)
    @printf("\n\n Yay!! 🙏")
end

Objective function value: 0.5000000 
Minimizing values: 0.5000000, 0.5000000

 Yay!! 🙏

## Pattern 2

ちょっと違う方法。
関数のリストを作って、`register` ではそのリストから関数をとってくる。
でも、結局 `register`は一つ一つの関数それぞれに使ってる。

In [55]:
model = Model(Ipopt.Optimizer);
set_silent(model)

@variable(model, x[i = 1:2]);

func_list = [
    (x...) -> x[1] + x[2] - 2.0,
    (x...) -> - x[1] + x[2] - 1.0,
    (x...) -> - x[1] -  x[2] + 1.0,
    (x...) -> x[1] - x[2] - 1.0
]

register(model, :fn1, 2, func_list[1]; autodiff = true);
register(model, :fn2, 2, func_list[2]; autodiff = true);
register(model, :fn3, 2, func_list[3]; autodiff = true);
register(model, :fn4, 2, func_list[4]; autodiff = true);

@NLconstraint(model, fn1(x...) <= 0.0);
@NLconstraint(model, fn2(x...) <= 0.0);
@NLconstraint(model, fn3(x...) <= 0.0);
@NLconstraint(model, fn4(x...) <= 0.0);

@NLobjective(model, Min, x[1]^2 + x[2]^2);

print(model)
optimize!(model)

objective_min = objective_value(model);
minx = value.(x)

@printf("Objective function value: %.7f \n", objective_min)
@printf("Minimizing values: %.7f, %.7f", minx[1], minx[2])

if (minx[1] ≈ 0.5) & (minx[2] ≈ 0.5)
    @printf("\n\n Yay!! 🙏")
end

Objective function value: 0.5000000 
Minimizing values: 0.5000000, 0.5000000

 Yay!! 🙏

## Pattern 3

`register` を一つ一つの関数に使うの面倒くさい。
というわけで、for loopを使います。
でも、`register` の2つ目の引数はSymbol、これどうすんの…？

というのに対処したのがこちら。

In [57]:
model = Model(Ipopt.Optimizer);
set_silent(model)

@variable(model, x[i = 1:2]);

func_list = [
    (x...) -> x[1] + x[2] - 2.0,
    (x...) -> - x[1] + x[2] - 1.0,
    (x...) -> - x[1] -  x[2] + 1.0,
    (x...) -> x[1] - x[2] - 1.0
]

N = length(func_list);

for i in 1:N
    register(model, Symbol("fn", i), 2, func_list[i]; autodiff = true);
end

@NLconstraint(model, fn1(x...) <= 0.0);
@NLconstraint(model, fn2(x...) <= 0.0);
@NLconstraint(model, fn3(x...) <= 0.0);
@NLconstraint(model, fn4(x...) <= 0.0);

@NLobjective(model, Min, x[1]^2 + x[2]^2);

print(model)
optimize!(model)

objective_min = objective_value(model);
minx = value.(x)

@printf("Objective function value: %.7f \n", objective_min)
@printf("Minimizing values: %.7f, %.7f", minx[1], minx[2])

if (minx[1] ≈ 0.5) & (minx[2] ≈ 0.5)
    @printf("\n\n Yay!! 🙏")
end

Objective function value: 0.5000000 
Minimizing values: 0.5000000, 0.5000000

 Yay!! 🙏

## Pattern 4

ここまで来たら `@NLconstraint` にも for loop 使いたい。
そうすれば、どんな長さの `func_list` にも対応できる。
でも、`register` では `fn1`、`fn2`...っていう名前で関数を定義している。
これを、JuMPのモデルの constraints を作る際にどうやって使う？

どうやら `add_nonlinear_constraint` っていうので、Juliaの `Expr`オブジェクトをJuMPの中で使えるそうな。
詳しくは[こちら](https://jump.dev/JuMP.jl/stable/manual/nlp/#Raw-expression-input)。

というわけで書いてみたのが以下の通り。

In [60]:
model = Model(Ipopt.Optimizer);
set_silent(model)

@variable(model, x[i = 1:2]);

func_list = [
    (x...) -> x[1] + x[2] - 2.0,
    (x...) -> - x[1] + x[2] - 1.0,
    (x...) -> - x[1] -  x[2] + 1.0,
    (x...) -> x[1] - x[2] - 1.0
]

N = length(func_list);

for i in 1:N
    register(model, Symbol("fn", i), 2, func_list[i]; autodiff = true);
end

for i in 1:N
    add_nonlinear_constraint(
        model, 
        :($(Symbol("fn", i))($(x...)) <= 0)
    )
end

@NLobjective(model, Min, x[1]^2 + x[2]^2);

print(model)
optimize!(model)

objective_min = objective_value(model);
🐨 = value.(x)

@printf("Objective function value: %.7f \n", objective_min)
@printf("Minimizing values: %.7f, %.7f", 🐨[1], 🐨[2])

if (🐨[1] ≈ 0.5) & (🐨[2] ≈ 0.5)
    @printf("\n\n Yay!! 🙏 ")
end

Objective function value: 0.5000000 
Minimizing values: 0.5000000, 0.5000000

 Yay!! 🙏 

## Pattern 4'

`add_nonlinear_constraint` に似たので `add_nonlinear_expression`っていうのもあって、これは constraint に限らず、expression を作れる。
これを使って以下のように制約をつくることもできるよ。

In [66]:
model = Model(Ipopt.Optimizer);
set_silent(model)

@variable(model, x[i = 1:2]);

func_list = [
    (x...) -> x[1] + x[2] - 2.0,
    (x...) -> - x[1] + x[2] - 1.0,
    (x...) -> - x[1] -  x[2] + 1.0,
    (x...) -> x[1] - x[2] - 1.0
]

N = length(func_list);

for i in 1:N
    register(model, Symbol("fn", i), 2, func_list[i]; autodiff = true);
end

expr_list = [add_nonlinear_expression(model, :($(Symbol("fn", i))($(x...)))) for i in 1:N]
@NLconstraint(model, [i = 1:N], expr_list[i] <= 0) # <------------ Here

@NLobjective(model, Min, x[1]^2 + x[2]^2);

print(model)
optimize!(model)

objective_min = objective_value(model);
🐨 = value.(x)

@printf("Objective function value: %.7f \n", objective_min)
@printf("Minimizing values: %.7f, %.7f", 🐨[1], 🐨[2])

if (🐨[1] ≈ 0.5) & (🐨[2] ≈ 0.5)
    @printf("\n\n Yay!! 🙏 ")
end

Objective function value: 0.5000000 
Minimizing values: 0.5000000, 0.5000000

 Yay!! 🙏 